## Importamos las librerias

In [ ]:
import requests
import pandas as pd
from datetime import datetime

- Creamos la función **`get_stablecoin_historical_data`** para hacer la solicitud a la *API de Coingecko* y extraer la información de las monedas estables seleccionadas, por un período de 14 días, a partir del 01-08-2023 hasta el 15-08-2023.

In [1]:
def get_stablecoin_historical_data(id, days):
    history_endpoint = f"https://api.coingecko.com/api/v3/coins/{id}/market_chart"
    history_params = {
        "vs_currency": "usd",
        "days": days,
        "interval": "daily"
    }
    
    history_response = requests.get(history_endpoint, params=history_params)
    
    if history_response.status_code == 200:
        history_data = history_response.json()
        timestamps = [entry[0] for entry in history_data["prices"]]
        prices = [entry[1] for entry in history_data["prices"]]
        market_caps = [entry[1] for entry in history_data["market_caps"]]
        total_volumes = [entry[1] for entry in history_data["total_volumes"]]
        
        dates = [datetime.fromtimestamp(ts / 1000).strftime('%Y-%m-%d') for ts in timestamps]
        
        df_historical = pd.DataFrame({
            "Date": dates,
            "Price": prices,
            "MarketCap": market_caps,
            "TotalVolume": total_volumes,
            "Stablecoin": [id] * len(timestamps)
        })
        
        return df_historical
    else:
        print(f"Error al obtener los datos históricos para {id}. Detalles del error:")
        print(history_response.text)
        return None

stable = ["tether", "usd-coin", "dai", "true-usd", "binance-usd", "frax", "usdd", "paxos-standard"]

days = "14"  

- Se crea el DataFrame **df_stable**

In [2]:
df_stable = pd.DataFrame()
for symbol in stable:
    df_symbol = get_stablecoin_historical_data(id=symbol, days=days)
    if df_symbol is not None:
        df_stable = pd.concat([df_stable, df_symbol], ignore_index=True)

- Se observa el contenido del DataFrame.

In [4]:
df_stable

,Date,Price,MarketCap,TotalVolume,Stablecoin
0,2023-08-01,0.999942,8.381631e+10,2.537704e+10,tether
1,2023-08-02,0.999242,8.388853e+10,2.364385e+10,tether
2,2023-08-03,0.998680,8.384106e+10,1.789421e+10,tether
3,2023-08-04,0.998912,8.371969e+10,1.129369e+10,tether
4,2023-08-05,0.998987,8.381698e+10,8.632620e+09,tether
...,...,...,...,...,...
115,2023-08-11,0.999782,5.061780e+08,1.544046e+06,paxos-standard
116,2023-08-12,1.000142,5.061009e+08,3.300889e+06,paxos-standard
117,2023-08-13,0.999494,5.058950e+08,2.275614e+06,paxos-standard
118,2023-08-14,0.999690,5.060368e+08,2.246954e+06,paxos-standard


- Renombramos 'Stablecoin' como 'Currency' para que sea más fácil unir después con el otro dataframe.

In [5]:
df_stable.rename(columns={'Stablecoin': 'Currency'}, inplace=True)

- Guardarmos DataFrame en un archivo CSV llamado **stable_data2.csv**

In [7]:
df_stable.to_csv("stable_data2.csv", index=False)